In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.tools import Tool
from langchain_classic.chains import RetrievalQA

from config import conf


# 1. RAG 설정: 관련 도메인 지식 문서 로드
# 예: "대학 중도탈락 방지 가이드북.pdf" 같은 관련 가이드 파일, 또는 내부 절차 문서
def setup_rag_tool(llm):
    # 문서 로드 (텍스트 또는 PDF)
    # loader = TextLoader("./input/educational_policy.txt")
    # documents = loader.load()

    # 임시 데이터로 예시 생성 TODO 실제 파일 경로로 대체
    example_text = """
    대학 중도 탈락의 주요 원인은 미복학, 자퇴, 학사경고 순입니다.
    특히 미복학 학생의 경우 심리적 거리감이 생겨 자퇴로 이어질 확률이 30% 더 높습니다.
    예방을 위해서는 휴학 기간 중 주기적인 학사 상담과 복학 장려 프로그램이 필수적입니다.
    """
    from langchain_core.documents import Document
    documents = [Document(page_content=example_text)]

    # 2. 텍스트 분할 (RecursiveCharacterTextSplitter)
    # 단락, 문장 단위로 분할하여 의미 보존 극대화
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,
        chunk_overlap=100,
        separators=["\n\n", "\n", " ", ""]
    )
    docs = text_splitter.split_documents(documents)

    # page_content가 비어있지 않고, 최소 2글자 이상인 것만 유지
    docs = [d for d in docs if d.page_content and len(d.page_content.strip()) > 1]
    
    if not docs:
        raise ValueError("임베딩할 유효한 문서 내용이 없습니다. 입력 데이터를 확인하세요.")

    # Ollama 로컬 임베딩 모델 사용, TODO conf.embedding_model 별개 모델이나 임베딩 전용 모델 찾아보기 e.g. mxbai-embed-realms, mxbai-embed-large 한국어 bge-m3
    embeddings = OllamaEmbeddings(model=conf.embedding_model)

    # VectorStore 생성 (FAISS)
    vectorstore = FAISS.from_documents(docs, embeddings)

    # Retrieval QA 체인 생성 TODO k값 조절로 검색 개수 최적화
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm, # 추론은 Gemma 3:27b 사용
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
    )

    # 에이전트가 사용할 도구로 변환
    rag_tool = Tool(
        name="domain_knowledge_search",
        func=rag_chain.invoke,
        description="대학 학업 중단, 교육 정책, 학생 지원 전략에 대한 도메인 지식을 검색할 때 사용하세요."
    )
    return rag_tool

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_ollama import OllamaLLM
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

from config import conf

# 1. 시각화 설정
%matplotlib inline

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="whitegrid", font='NanumGothic') # Seaborn 테마 설정

# 2. 분석할 CSV 파일 로드 
file_path = "./input/중도탈락 학생 현황 (대학)_2026-01-14175237927.csv" #
df = pd.read_csv(file_path, encoding='utf-8')  # 'cp949')

print("데이터 로드 완료")
display(df.head()) 
print(len(df))

# 3. 모델 및 에이전트 설정
model_name = conf.MODEL_ID

llm = OllamaLLM(model=model_name, 
                temperature=0,
                # 모델이 한 번의 응답에서 생성할 최대 토큰 수 (설명 길이를 강제 제한) 길게 설명하는 것을 방지
                num_predict=2048,  # # 512,  # 이상한 소리하면 줄여보기
               )
llm_with_stop = llm.bind(stop=["Observation:", "\nObservation", "Thought:", "\n\tObservation"])  # 중지 시퀀스 설정
# 에이전트가 지켜야 할 철저한 규칙 정의
custom_prefix = conf.CUSTOM_PREFIX + """

[Current Dataframe Info]
- Variable Name: df
- Columns: {col_list}
""".format(col_list=list(df.columns))

# RAG 도구 생성
knowledge_tool = setup_rag_tool(llm)

agent = create_pandas_dataframe_agent(
    llm_with_stop,  # llm,
    df,
    verbose=True,               # Thought/Action/Observation 실시간 출력 (custom_prefix, agent_executor_kwargs, full_query 등 tuning 위한 debugger로 활용
    # return_intermediate_steps=False, # 중간 단계 생략 시도
    allow_dangerous_code=True,  # 코드 실행 허용
    extra_tools=[knowledge_tool],  # # extra_tools로 RAG 추가
    # 실행 엔진(AgentExecutor)에 전달할 인자를 별도 딕셔너리로 분리 (에러 발생 시 모델에게 던져줄 가이드 문구)
    agent_executor_kwargs={
        "handle_parsing_errors": conf.HANDLE_PARSING_ERRORS,
    },
    agent_type="zero-shot-react-description",
    max_iterations=100, # 파싱 에러 시 재시도 횟수
    prefix=custom_prefix,
)

# 4. 분석 상세 요구 사항 입력 TODO 테스트용
summary_req = "전체적인 통계값 요약해줘"
hypo_req = "학업중단 사유별 학업중단율을 비교하고 가설을 설정하여라"
result_req = "가설을 검증하기 위해 실제 코드를 실행한 통계적 근거를 보여줘"
biz_req = "분석 결과를 바탕으로 한 분석 전략을 제안해줘"

# 5. 쿼리 조합
full_query = f"""
아래 요청사항에 맞춰 데이터프레임을 분석하고 한국어로 보고서를 작성하세요:

1. 데이터 요약: {summary_req}
2. 가설 설정: {hypo_req}
3. 분석 결과: {result_req}
4. 비즈니스 제언: {biz_req}

반드시 파이썬 코드를 실행하여 얻은 통계 수치를 바탕으로 답변하세요.
""" + conf.FULL_QUERY

# 6. 실행
print(f"{model_name} AGENT GO!GO!GO!...\n")
try:
    response = agent.invoke(full_query)

    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")

데이터 로드 완료


,기준연도,학교종류,설립구분,지역,상태,학교,재적학생\n(A),계(B),미등록,미복학,...,계(B`),미등록.1,미복학.1,자퇴.1,학사경고.1,학생활동.1,유급제적.1,재학연한초과.1,기타.1,중도탈락학생(신입생)비율(%)\n(B`/A`) × 100
0,2024,대학교,사립,경남,기존,가야대학교(김해),"2,012",179,14,30,...,42,2,0,40,0,0,0,0,0,11.4
1,2024,대학교,사립,경기,기존,가천대학교,"28,264","1,090",133,117,...,451,14,0,437,0,0,0,0,0,8.6
2,2024,대학교,사립,강원,기존,가톨릭관동대학교,"8,410",627,4,183,...,127,1,0,126,0,0,0,0,0,10.8
3,2024,대학교,사립,충북,기존,가톨릭꽃동네대학교,557,26,0,0,...,18,0,0,14,0,0,0,0,4,15.9
4,2024,대학교,사립,경기,기존,가톨릭대학교,"9,794",384,15,61,...,164,3,0,161,0,0,0,0,0,8.8


246
gemma3:12b AGENT GO!GO!GO!...



> Entering new AgentExecutor chain...


/tmp/ipykernel_838176/2137011121.py:82: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = agent.run(full_query)


Thought: The user wants a comprehensive data analysis report in Korean, including summary statistics, hypothesis testing regarding dropout rates by reason, code execution for statistical evidence, and business recommendations. I need to perform the analysis and present the results in the specified format.

Action: python_repl_ast
Action Input: ```python
import pandas as pd
import numpy as np
import scipy.stats as stats

# Assuming df is already loaded

# 1. 데이터 요약
print("1. 데이터 요약:")
print(df.describe(include='all'))

# 2. 가설 설정
# 가설: 학업중단 사유별 학업중단율에 차이가 있을 것이다.
# 귀무가설: 학업중단 사유별 학업중단율에 차이가 없다.

# 3. 분석 결과
# 학업중단 사유별 중도탈락학생비율(%)을 비교하기 위해 ANOVA를 사용한다.
# 먼저, 각 학업중단 사유별 비율을 계산한다.
drop_out_reasons = ['자퇴', '유급제적', '재학연한초과', '기타', '중도탈락학생비율(%)']
drop_out_rates = df[drop_out_reasons].copy()

# 중도탈락학생비율(%) 컬럼을 numeric으로 변환
df['중도탈락학생비율(%)'] = pd.to_numeric(df['중도탈락학생비율(%)'], errors='coerce')

# ANOVA 수행
f_statistic, p_value = stats.f_oneway(df['자퇴'], df['유급제적'], df['재학연한초과'], df['기타'], df['중도탈